In [ ]:
import shutil
from experiment import Experiment

In [ ]:
# Unzip files
shutil.unpack_archive("data_export.zip", "data")

In [ ]:
# Process files to create an Experiment
experiment = Experiment.reconstruct_experiment_from_folder("data/montreal-tamper")

print(experiment.get_experiment_info())

# baseline_experiment = Experiment.reconstruct_experiment_from_folder("data/simulator-delay-12-weird")
baseline_experiment = None
if baseline_experiment:
    print(baseline_experiment.get_experiment_info())

In [ ]:
# Test 1 - honest delete
print(experiment.run_test_1())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_1())

In [ ]:
# Test 2 - decrypt
print(experiment.run_test_2())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_2())

In [ ]:
# Test 3 - honest delete and then decrypt
print(experiment.run_test_3())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_3())

In [ ]:
# Test 4 - malicious delete and then decrypt
print(experiment.run_test_4())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_4())

In [ ]:
# Test 5 - tamper detection
print(experiment.run_test_5())

if baseline_experiment:
    print("\nBaseline experiment:")
    print(baseline_experiment.run_test_5())